# GA Data Science 16 (DAT16) - Lab 15

### Neural Networks

Francesco Mosconi, Justin Breucop

### Today

Implementing a Neural Network from Scratch

This lab is inspired by [this](http://www.wildml.com/2015/09/implementing-a-neural-network-from-scratch/) blog post

In [ ]:
# Package imports
import numpy as np
import sklearn
import sklearn.datasets
import sklearn.linear_model

# Display plots inline and change default figure size
from bokeh.plotting import figure,gridplot,show,output_notebook
from bokeh.models import Range1d
output_notebook()

## Generating a dataset

Let's start by generating a dataset we can play with. Fortunately, [scikit-learn](http://scikit-learn.org/) has some useful dataset generators, so we don't need to write the code ourselves. We will go with the [`make_moons`](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html) function.

In [ ]:
# Generate a dataset and plot it
np.random.seed(0)
X, y = sklearn.datasets.make_moons(200, noise=0.20)

p = figure(tools='', title="Two half moon distributed datasets")
color_dict = {0:'blue',1:'red'}
y_colors = [color_dict[label] for label in y]

# These are glyphs
p.circle(X[:,0], X[:,1],size=10, color=y_colors, alpha = 0.5)

# show the results
show(p)

The dataset we generated has two classes, plotted as red and blue points. You can think of the blue dots as male patients and the red dots as female patients, with the x- and y- axis being medical measurements. 

Our goal is to train a Machine Learning classifier that predicts the correct class (male of female) given the x- and y- coordinates. Note that the data is not *linearly separable*, we can't draw a straight line that separates the two classes. This means that linear classifiers, such as Logistic Regression, won't be able to fit the data unless you hand-engineer non-linear features (such as polynomials) that work well for the given dataset.

In fact, that's one of the major advantages of Neural Networks. You don't need to worry about [feature engineering](http://machinelearningmastery.com/discover-feature-engineering-how-to-engineer-features-and-how-to-get-good-at-it/). The hidden layer of a neural network will learn features for you.

## Logistic Regression

To demonstrate the point let's train a Logistic Regression classifier. It's input will be the x- and y-values and the output the predicted class (0 or 1). To make our life easy we use the Logistic Regression class from `scikit-learn`.

In [ ]:
# Train the logistic rgeression classifier
clf = sklearn.linear_model.LogisticRegressionCV()
clf.fit(X, y)

In [ ]:
# Helper function to plot a decision boundary.
# If you don't fully understand this function don't worry, it just generates the contour plot below.
def plot_decision_boundary(pred_func, point_size = 10):
    # Set min and max values and give it some padding
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5

    # Generate a grid of points with distance h between them
    h = 0.01
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    # Predict the function value for the whole gid
    Z = pred_func(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    control_row = np.ones(Z.shape[1])*Z.max()+0.001
    #NOTE: control row adds a max value outside of the grid to correct colors. 
    # Don't worry about explaining control_row
    Z = np.vstack((Z,control_row))


    p1 = figure(tools='',
                x_range=(xx.min(), xx.max()), 
                y_range=(yy.min(), yy.max()))

    # Plot the contour and training examples
    p1.image(image=[Z], x=[xx.min()], y=[yy.min()],
             dw=[xx.max()-xx.min()], dh=[yy.max()-yy.min()],
             palette='RdYlBu11')

    p1.circle(X[:, 0], X[:, 1], 
              color=y_colors,
              line_color='black',
              size=point_size,alpha = 0.7)
    return(p1)

In [ ]:
# Plot the decision boundary
p1 = plot_decision_boundary(lambda x: clf.predict(x))
p1.title = "Logistic Regression"
show(p1)

The graph shows the decision boundary learned by our Logistic Regression classifier. It separates the data as good as it can using a straight line, but it's unable to capture the "moon shape" of our data.

### Implementation of the Neural Network

Now we are ready for our implementation. We start by defining some useful variables and parameters for gradient descent:

In [ ]:
num_examples = len(X) # training set size
nn_input_dim = 2 # input layer dimensionality
nn_output_dim = 2 # output layer dimensionality

# Gradient descent parameters (I picked these by hand)
epsilon = 0.01 # learning rate for gradient descent
reg_lambda = 0.01 # regularization strength 

First let's implement the loss function we defined above. We use this to evaluate how well our model is doing:

In [ ]:
# Exercise: add comments to each line of the next function
def calculate_loss(model):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    z1 = X.dot(W1) + b1
    a1 = np.tanh(z1)
    z2 = a1.dot(W2) + b2
    exp_scores = np.exp(z2)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    corect_logprobs = -np.log(probs[range(num_examples), y])
    data_loss = np.sum(corect_logprobs)
    data_loss += reg_lambda/2 * (np.sum(np.square(W1)) + np.sum(np.square(W2)))
    return 1./num_examples * data_loss

We also implement a helper function to calculate the output of the network. It does forward propagation and returns the class with the highest probability.

In [ ]:
# Exercise: add comments to each line of the next function
def predict(model, x):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    z1 = x.dot(W1) + b1
    a1 = np.tanh(z1)
    z2 = a1.dot(W2) + b2
    exp_scores = np.exp(z2)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    return np.argmax(probs, axis=1)

Finally, here comes the function to train our Neural Network. It implements batch gradient descent using the backpropagation derivates we found above.

In [ ]:
# Exercise: comments the blocks of code with high level description
def build_model(nn_hdim, num_passes=20000, print_loss=False):
    # block ...
    np.random.seed(0)
    W1 = np.random.randn(nn_input_dim, nn_hdim) / np.sqrt(nn_input_dim)
    b1 = np.zeros((1, nn_hdim))
    W2 = np.random.randn(nn_hdim, nn_output_dim) / np.sqrt(nn_hdim)
    b2 = np.zeros((1, nn_output_dim))

    # block ...
    model = {}
    
    # block ...
    for i in xrange(0, num_passes):

        # block ...
        z1 = X.dot(W1) + b1
        a1 = np.tanh(z1)
        z2 = a1.dot(W2) + b2
        exp_scores = np.exp(z2)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

        # block ...
        delta3 = probs
        delta3[range(num_examples), y] -= 1
        dW2 = (a1.T).dot(delta3)
        db2 = np.sum(delta3, axis=0, keepdims=True)
        delta2 = delta3.dot(W2.T) * (1 - np.power(a1, 2))
        dW1 = np.dot(X.T, delta2)
        db1 = np.sum(delta2, axis=0)

        # block ...
        dW2 += reg_lambda * W2
        dW1 += reg_lambda * W1

        # block ...
        W1 += -epsilon * dW1
        b1 += -epsilon * db1
        W2 += -epsilon * dW2
        b2 += -epsilon * db2
        
        # block ...
        model = { 'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}
        
        # block ...
        if print_loss and i % 1000 == 0:
            print "Loss after iteration %i: %f" %(i, calculate_loss(model))
    
    return model

### A network with a hidden layer of size 3

Let's see what happens if we train a network with a hidden layer size of 3.


In [ ]:
# Build a model with a 3-dimensional hidden layer
model = build_model(3, print_loss=True)

# Plot the decision boundary
p2 = plot_decision_boundary(lambda x: predict(model, x))
p2.title = "Decision Boundary for hidden layer size 3"
show(p2)

Yay! This looks pretty good. Our neural networks was able to find a decision boundary that successfully separates the classes.

# Varying the hidden layer size

In the example above we picked a hidden layer size of 3. Let's now get a sense of how varying the hidden layer size affects the result.


In [ ]:
hidden_layer_dimensions = [1, 2, 3, 4, 5, 20, 30, 50]

row = []

for i, nn_hdim in enumerate(hidden_layer_dimensions):
    model = build_model(nn_hdim)
    p = plot_decision_boundary(lambda x: predict(model, x), point_size=8)
    p.title = 'Hidden Layer size %d' % nn_hdim
    p.plot_width=360
    p.plot_height=360
    row.append(p)

grid = np.array(row).reshape(len(row)/2, 2)
show(gridplot(grid.tolist()))

We can see that while a hidden layer of low dimensionality nicely capture the general trend of our data, but higher dimensionalities are prone to overfitting. They are "memorizing" the data as opposed to fitting the general shape. If we were to evaluate our model on a separate test set (and you should!) the model with a smaller hidden layer size would likely perform better because it generalizes better. We could counteract overfitting with stronger regularization, but picking the a correct size for hidden layer is a much more "economical" solution.

# Exercises

## Exercise 1

Encapsulate the gradient calculation in a function:

    def evaluate_gradient(X, y, l, W1, b1, W2, b2):
    """
    l : len(y) is the number of data points
    """

modify the build_model function to take the newly defined function as parameter

    def build_model_g_func(nn_hdim, num_passes=20000, print_loss=False, gradient_func=evaluate_gradient):


In [ ]:
def evaluate_gradient(X, y, l, W1, b1, W2, b2):
    # your code here....
    
    return dW1, db1, dW2, db2

def build_model_g_func(nn_hdim, num_passes=20000, print_loss=False, gradient_func=evaluate_gradient):
    # your code here....
    
    return model

Check that you still get the same results

In [ ]:
# Build a model with a 3-dimensional hidden layer
model = build_model_g_func(3, print_loss=True)

# Plot the decision boundary
p2 = plot_decision_boundary(lambda x: predict(model, x))
p2.title = "Decision Boundary for hidden layer size 3"
show(p2)

## Exercise 2

Instead of batch gradient descent, use minibatch gradient descent ([more info](http://cs231n.github.io/optimization-1/#gd)) to train the network. Minibatch gradient descent typically performs better in practice.

Define a new function:

    def build_model_sample(nn_hdim, num_passes=20000, print_loss=False,
                       gradient_func=evaluate_gradient, sample_size = 32):

and test it on the same data

In [ ]:
def build_model_sample(nn_hdim, num_passes=20000, print_loss=False,
                       gradient_func=evaluate_gradient, sample_size = 32):
    
    # your code here....
    
    return model

# Build a model with a 3-dimensional hidden layer
model = build_model_sample(3, print_loss=True)

# Plot the decision boundary
p2 = plot_decision_boundary(lambda x: predict(model, x))
p2.title = # set an appropriate title....
show(p2)

## Exercise 3

We used a fixed learning rate $\epsilon$ for gradient descent. Implement an annealing schedule for the gradient descent learning rate ([more info](http://cs231n.github.io/neural-networks-3/#anneal)). 

Modify the build_model function to 

    def build_model_anneal(nn_hdim, num_passes=20000, print_loss=False,
                            gradient_func=evaluate_gradient):
                            
to allow for annealing.

In [ ]:
def build_model_anneal(nn_hdim, num_passes=20000, print_loss=False,
                       gradient_func=evaluate_gradient):

    # your code here....
    
    return model

# Build a model with a 3-dimensional hidden layer
model = build_model_anneal(3, print_loss=True)

# Plot the decision boundary
p2 = plot_decision_boundary(lambda x: predict(model, x))
p2.title = # title...
show(p2)

## Exercise 4

We used a $\tanh$ activation function for our hidden layer. Experiment with other activation functions (some are mentioned above). Note that changing the activation function also means changing the backpropagation derivative.

You'll need to define the following functions:

    def calculate_loss_sigmoid(model):

    def evaluate_gradient_sigmoid(X, y, l, W1, b1, W2, b2):

    def build_model_sigmoid(nn_hdim, num_passes=20000, print_loss=False,
                       gradient_func=evaluate_gradient_sigmoid, sample_size = 32):



In [ ]:
def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))

# Helper function to evaluate the total loss on the dataset
def calculate_loss_sigmoid(model):

    # your code here ...
    
    return 1./num_examples * data_loss


def evaluate_gradient_sigmoid(X, y, l, W1, b1, W2, b2):
    
    # your code here ...
    
    return dW1, db1, dW2, db2

def build_model_sigmoid(nn_hdim, num_passes=20000, print_loss=False,
                       gradient_func=evaluate_gradient_sigmoid, sample_size = 32):

    # your code here ...
        
    return model

# Helper function to predict an output (0 or 1)
def predict_sigmoid(model, x):
    
    # your code here ...
    
    return np.argmax(probs, axis=1)

# Build a model with a 3-dimensional hidden layer
model = build_model_sigmoid(3, print_loss=True)

# Plot the decision boundary
p2 = plot_decision_boundary(lambda x: predict_sigmoid(model, x))
p2.title = # title...
show(p2)

## Exercise 5

Extend the network from two to three classes. You will need to generate an appropriate dataset for this.

In [ ]:
# Generate a dataset and plot it
np.random.seed(0)
X, y = sklearn.datasets.make_classification(300, n_classes = 3, n_features=2, n_redundant=0,
                                            n_clusters_per_class = 1)

p = figure(tools='', title="Two half moon distributed datasets")
color_dict = {0:'blue',1:'yellow', 2:'red'}
y_colors = [color_dict[label] for label in y]

# These are glyphs
p.circle(X[:,0], X[:,1],size=10, color=y_colors, alpha = 0.5)

# show the results
show(p)

Modify the global parameters of the network and re-define the following functions to allow for 3 class dataset:

    def build_model(nn_hdim, nn_input_dim=2, nn_output_dim=2, num_passes=20000, print_loss=False):

    

In [ ]:
num_examples = 300

def build_model(nn_hdim, nn_input_dim=2, nn_output_dim=2, num_passes=20000, print_loss=False):
    
    # your code here ...
    
    return model


# Build a model with a 3-dimensional hidden layer
model = build_model(3, 2, 3, 300, print_loss=True)

# Plot the decision boundary
p2 = plot_decision_boundary(lambda x: predict(model, x))
p2.title = # title ... 
show(p2)

## Exercise 6

Extend the network to four layers. Experiment with the layer size. Adding another hidden layer means you will need to adjust both the forward propagation as well as the backpropagation code.

In [ ]:
# Helper function to evaluate the total loss on the dataset
def calculate_loss(model):
    
    # your code here ...

    return 1./num_examples * data_loss

# Helper function to predict an output (0 or 1)
def predict(model, x):
    
    # your code here ...
    
    return np.argmax(probs, axis=1)

def build_model(nn_hdim1, nn_hdim2, nn_input_dim, nn_output_dim, num_examples, num_passes=20000, print_loss=False):
    
    # your code here ...
    
    return model


# Build a model with a 3-dimensional hidden layer
model = build_model(4, 3, 2, 3, 300, print_loss=True)


# Plot the decision boundary
p2 = plot_decision_boundary(lambda x: predict(model, x))
p2.title = # title ...
show(p2)